<a href="https://colab.research.google.com/github/changyuhsin1999/Fake_Medical_News_Detection_Tool/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "Fake_Medical_News_Detection_Tool" # Enter repo name
git_path = 'https://github.com/changyuhsin1999/Fake_Medical_News_Detection_Tool.git'
!git clone "{git_path}"

Cloning into 'Fake_Medical_News_Detection_Tool'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 247 (delta 47), reused 0 (delta 0), pack-reused 160
Receiving objects: 100% (247/247), 5.22 MiB | 9.88 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [31]:
import numpy as np
import pandas as pd
import nltk
import torch
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
raw_df = pd.read_csv('/content/Fake_Medical_News_Detection_Tool/data/raw_df.csv')
tokenizer = Tokenizer(num_words = 3000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = raw_df['full_text'])
X = tokenizer.texts_to_sequences(texts = raw_df['full_text'])
X = pad_sequences(sequences = X, maxlen = 3000, padding = 'pre')
y = raw_df['label'].values

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 101)

In [41]:
# Initiate the model
lstm_model = Sequential(name = 'lstm_nn_model')
lstm_model.add(layer = Embedding(input_dim = 3000, output_dim = 120, name = '1st_layer'))
lstm_model.add(layer = LSTM(units = 120, dropout = 0.2, recurrent_dropout = 0.2, name = '2nd_layer'))
lstm_model.add(layer = Dropout(rate = 0.5, name = '3rd_layer'))
lstm_model.add(layer = Dense(units = 120,  activation = 'relu', name = '4th_layer'))
lstm_model.add(layer = Dropout(rate = 0.5, name = '5th_layer'))
lstm_model.add(layer = Dense(units = len(set(y)),  activation = 'sigmoid', name = 'output_layer'))
# compiling the model
lstm_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [43]:
lstm_model_fit = lstm_model.fit(X_train, y_train, epochs = 1)

80/80 [==============================] - 1185s 15s/step - loss: 0.0902 - accuracy: 0.9682


In [54]:
y_pred = (lstm_model.predict(X_test) >= 0.5).astype("int")
y_pred = y_pred[:,1]

20/20 [==============================] - 17s 877ms/step


In [55]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       205
           1       0.99      0.99      0.99       432

    accuracy                           0.99       637
   macro avg       0.98      0.99      0.99       637
weighted avg       0.99      0.99      0.99       637

